In [0]:
# !pip install 'keras==2.2.0'
# !pip install 'numpy==1.16.1'
# !pip install 'tensorflow==1.7.0'
# !pip install 'tensorflow_hub==0.1.1'

In [0]:
import torch
dev = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)
print(dev)

cuda


In [0]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!unzip sms-spam-collection-dataset.zip

sms-spam-collection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sms-spam-collection-dataset.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

MAX_LEN = 128
def make_clean(s) :
  for i in range(10) :
    s = s.replace(str(i), ' ')
  tokens = np.array(tokenizer.tokenize(s.lower()))
  tokens = tokens[~np.isin(tokens, stopwords.words())]
  return ' '.join(tokens)

df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df['clean'] = df['v2'].progress_apply(make_clean)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from utils import DataKeeper

dk = DataKeeper(df.clean.tolist(), (df.v1=='ham').astype(int).tolist())
(x_train, y_train), (x_test, y_test) = dk.load_data(MAX_LEN)

Using TensorFlow backend.


In [0]:
import tensorflow as tf
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from elmo import ELMoEmbedding

idx2word = dk.get_idx2word()

print('Build model...')
sentence_input = Input(shape=(x_train.shape[1],), dtype=tf.int64)
sentence_embedding = ELMoEmbedding(idx2word=idx2word, output_mode="elmo", trainable=True)(sentence_input) # These two are interchangeable
pooled = GlobalMaxPooling1D()(sentence_embedding)
hidden = Dense(1024, activation='relu')(pooled)
output = Dense(1, activation='softmax')(hidden)

# sentence_input = Input(shape=(x_train.shape[1],), dtype=tf.int64)
# sentence_embedding = ELMoEmbedding(idx2word=idx2word, output_mode="elmo", trainable=True)(sentence_input) # These two are interchangeable
# convolution = Convolution1D(50, 3, padding='same', activation='relu')(sentence_embedding)
# convolution = GlobalMaxPooling1D()(convolution)
# dropout = Dropout(0.5)(convolution)
# hidden = Dense(50, activation='relu')(dropout)
# output = Dense(1, activation='sigmoid')(hidden)

model = Model(inputs=sentence_input, outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

Build model...
INFO:tensorflow:Initialize variable module_6/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module_6/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights
INFO:tensorflow:Initialize variable module_6/bilm/CNN/W_cnn_0:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module_6/bilm/CNN/W_cnn_1:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_1
INFO:tensorflow:Initialize variable module_6/bilm/CNN/W_cnn_2:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_2
INFO:tensorflow:Initialize variable module_6/bil

In [0]:
import keras
import numpy as np
import sklearn.metrics as sklm
import json


class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]

        self.loss = keras.losses.binary_crossentropy(targ, predict)
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        json.dump({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }, open("record_of_epoch_"+str(len(self.loss)), 'w'))

        return

metrics = Metrics()

In [0]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), callbacks=[metrics])

Train on 4458 samples, validate on 1114 samples
Epoch 1/5
4458/4458 [==============================] - 2001s 449ms/step - loss: 2.0670 - acc: 0.8703 - val_loss: 2.4185 - val_acc: 0.8483


TypeError: ignored

In [25]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), callbacks=[metrics])

Train on 4458 samples, validate on 1114 samples
Epoch 1/5
4458/4458 [==============================] - 1992s 447ms/step - loss: 2.0670 - acc: 0.8703 - val_loss: 2.4185 - val_acc: 0.8483


TypeError: ignored